In [1]:
import os
import time
import re
import chromedriver_autoinstaller
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm

In [2]:
chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito") # 시크릿 모드
# chrome_options.add_argument("--headless") # 리눅스 GUI 없는 디스플레이 모드
chrome_options.add_argument("--no-sandbox") # 리소스에 대한 엑서스 방지
chrome_options.add_argument("--disable-setuid-sandbox") # 크롬 충돌 방지
chrome_options.add_argument("--disable-dev-shm-usage") # 메모리 부족 에러 방지
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

try: # 크롬 드라이버
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)   
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)

/var/folders/5v/6byb3xz548v1xd0b83z24ys40000gn/T/ipykernel_2300/2948837750.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)
/var/folders/5v/6byb3xz548v1xd0b83z24ys40000gn/T/ipykernel_2300/2948837750.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)


In [10]:
gu_list = ['마포구','서대문구','은평구','종로구','중구','용산구','성동구','광진구',
           '동대문구','성북구','강북구','도봉구','노원구','중랑구','강동구','송파구',
           '강남구','서초구','관악구','동작구','영등포구','금천구','구로구','양천구','강서구']

In [3]:
driver.get('https://map.kakao.com/')

In [4]:
search_area = driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]') # 검색 창
search_area.send_keys("부평구 맛집")
search_area.send_keys("\n")

In [5]:
SearchPlace = driver.find_element(By.ID, "info.search.place")

In [6]:
SearchPlaceList = SearchPlace.find_element(By.ID, "info.search.place.list")

In [7]:
SearchPlaceListLi = SearchPlaceList.find_elements(By.TAG_NAME, "li")

In [8]:
CommentURL = []

In [9]:
for li in SearchPlaceListLi:
    DivName = li.find_elements(By.CLASS_NAME, "rating")
    for div in DivName:
        CommentURL.append(div.find_element(By.TAG_NAME, "a").get_attribute("href"))

In [10]:
driver.get(CommentURL[0])

In [50]:
while True:
    EvaluationReview = driver.find_element(By.CLASS_NAME, "evaluation_review")
    TotalReviews = int(driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[2]/div[8]/strong[1]/span").text)
    temp = EvaluationReview.find_element(By.CLASS_NAME, "link_more")
    temp_text = temp.text
    if temp_text == "후기 접기":
        break
    action = webdriver.ActionChains(driver)
    action.move_to_element(temp).perform()
    temp.send_keys("\n")

In [59]:
ReviewUl = EvaluationReview.find_element(By.TAG_NAME, "ul")
ReviewLi = ReviewUl.find_elements(By.TAG_NAME, "li")

In [96]:
temper = []
for li in ReviewLi:
    temper.append(li.text)

In [98]:
tmp = list(filter(None, temper))

In [126]:
tmp=driver.page_source
tmp2 = BeautifulSoup(tmp, "html.parser")

In [127]:
t3=tmp2.find(name="div", attrs={"class":"evaluation_review"})

In [114]:
star=t3.find_all('span',{'class':'ico_star inner_star'})

In [119]:
review = t3.find_all('p',{'class':'txt_comment'})

In [129]:
reviews = t3.find_all('p',{'class':'txt_comment txt_fold'})

In [131]:
for i in reviews:
    print(i.text)

진심 타지역에서 모임떄문에 간곳이지만 가격만 더럽게 비싸고... 직원은많은데 고기주문을해도오질않아서 물어보니 주문안들어가있어서 늦게사 오고 술도 주문하니 술도안와서 두세번 말해야돼고막판엔 후식으로 물냉이나 하나먹을까해서 시켯떠니 아니나 다를까 또 오지를않네...직원이 많은데도주문은 안들어가는게 참 난감하기도 짝이없고 나이좀있으신 아줌마분인거같은데 관리자급인지 먼지는모르겠으나 손님이 물냉하나시켰는데 물냉하고 비냉시키지 않았냐며 주문도 제데로 넣지도못하면서 무슨 어거지를 쓰는건지 손님상대로 어거지를 쓰는데...그것도 두세번 자기가 잘났다고따지면서 욱여돼는데...정말 어이없고 당혹스럽기 짝이없더군요...주문이나 제대로 받으면서 그런행동하든가...진심 어이업고 가격만비싸고 싸가지없어서 다신안갈듯더보기
규모도 크고 분위기도 좋아보여서 오리 한마리 먹으러 갔는데 블로거 리뷰에는 직원이 친절하다는글 많던데 코로나라 다 잘린모양..주문도 두번 세번 못 알아들어서 잘 못 가져오고 혼자 하시느라 바쁘겠지만 테이블에 사람도 별로 없는데 고기 한번을 안 뒤집어줌..이건 상관없는데 다른 사람들 리뷰엔 다 해준다고 보고나선지 왠지 빈정상함 그리고 솔직히 고기도 5만원치고 양이 진짜 적고 이 가격에 굳이..?먹으러 올만하진않을듯더보기
이거 맛있다고 하는 애들은 혀가 어떻게 됐나? 테이블 낮고 뜨겁고.... 밑반찬도 거의 없고... 볶음밥도 노맛....직원들도 그닥 친절하지 않음 대체 뭐가 장점인지 모르겠음...부평에 그렇게 먹을게 없나?다시는 안갈듯더보기


In [125]:
for i in review:
    print(i.text)

돈 개아까움 서비스 맛 뭐하나 빠지는거없이 쓰레기 홀직원 싸가지 없고더보기
안 좋다고 하는 것들이 진짜네요 좋은 건 주차하기 좋은 거더보기
알파카에 놀이방에 각종 고기들에 디저트까지... 색다르고 좋은 식당이였지만, 술먹고 진상짓하는 사람들이 좀 보였다..더보기
더보기
모든게 다 만족합니다~ 강강추 합니다퍼펙트~아주좋아요~~야간 여성 직원분 친철해요^^더보기
24시라 좋구 김치가 맛있어요 동치미도더보기
직원분들도 친절하시고 고기가 질이 좋아요 질기지 않아서 아이들도 맛있게 먹었습니다 ㅎㅎ 다음에 또 오고 싶어요더보기
불친절한 서비스, 순서없이 음식나오기, 시키면 함흥차사더보기
오리고기가 좀 질기긴하네여더보기
더보기
10년동안 단골이었어요 기존에 있던 직원분들은 다 안보이시고새로운 직원분들 모두 응대하는게 참.. 상이 너무 지저분해서 덜 닦인것같다고하니까물티슈로 가져다가 알아서 닦으랍니다잘못들은줄알았네요 ㅡ.ㅡ직원 바뀌고 설거지상태 셀프바 모두 엉망이에요더보기
역시 존맛탱 인정!!더보기
바베큐 제대로 경험할 수 있어요.더보기
더보기
맛있어요더보기
장소는 달라도 바베큐는 맛있어요더보기
나름 괜찮았는데 바베큐가 굿입니당~더보기
전 맛있었어요~더보기
진심 타지역에서 모임떄문에 간곳이지만 가격만 더럽게 비싸고... 직원은많은데 고기주문을해도오질않아서 물어보니 주문안들어가있어서 늦게사 오고 술도 주문하니 술도안와서 두세번 말해야돼고막판엔 후식으로 물냉이나 하나먹을까해서 시켯떠니 아니나 다를까 또 오지를않네...직원이 많은데도주문은 안들어가는게 참 난감하기도 짝이없고 나이좀있으신 아줌마분인거같은데 관리자급인지 먼지는모르겠으나 손님이 물냉하나시켰는데 물냉하고 비냉시키지 않았냐며 주문도 제데로 넣지도못하면서 무슨 어거지를 쓰는건지 손님상대로 어거지를 쓰는데...그것도 두세번 자기가 잘났다고따지면서 욱여돼는데...정말 어이없고 당혹스럽기 짝이없더군요...주문이나 제대로 받으면서 그런행동하든가...진심 어이업고 가격만비싸고 싸가지없어서 다신안갈듯더보기
더보기
왜이렇게 불친절해요?

In [83]:
temper = [v for v in temper if v != [''] and len(v) > 5]

[['시우애비', '후기', '12', '별점평균', '3.8', '2023.01.25.'],
 ['주하니', '후기', '10', '별점평균', '2.5', '2022.12.26.'],
 ['.', '후기', '23', '별점평균', '3.6', '2022.11.05.'],
 ['황신하', '후기', '102', '별점평균', '2.2', '2022.11.04.'],
 ['리~', '후기', '3', '별점평균', '4.7', '2022.10.17.'],
 ['추천포인트', '맛', '가성비', '친절', '분위기', '주차'],
 ['한냥충🐛', '후기', '139', '별점평균', '4.1', '2022.10.04.'],
 ['로라하', '후기', '101', '별점평균', '4.6', '2022.09.22.'],
 ['추천포인트', '맛', '가성비', '친절', '분위기', '주차'],
 ['Au revoir paris', '후기', '1', '별점평균', '1.0', '2022.09.10.'],
 ['만물박사', '후기', '32', '별점평균', '3.5', '2022.08.20.'],
 ['ㄱㅎㄱ', '후기', '1,051', '별점평균', '5.0', '2022.08.11.'],
 ['꼬르륵', '후기', '6', '별점평균', '3.0', '2022.06.06.'],
 ['10년동안 단골이었어요 ',
  '기존에 있던 직원분들은 다 안보이시고',
  '새로운 직원분들 모두 응대하는게 참.. ',
  '상이 너무 지저분해서 덜 닦인것같다고하니까',
  '물티슈로 가져다가 알아서 닦으랍니다',
  '잘못들은줄알았네요 ㅡ.ㅡ',
  '직원 바뀌고 설거지상태 셀프바 모두 엉망이에요',
  '더보기',
  '좋아요 3'],
 ['ssk', '후기', '87', '별점평균', '4.2', '2022.03.10.'],
 ['kos4', '후기', '141', '별점평균', '4.3', '2022.03.10.'],
 ['knh', '후기', '109', '

In [54]:
Clean_DataFrame = pd.DataFrame()

In [ ]:
temper = [v for v in temper if v != [''] and len(v) > 5]
for tmp in temper:
    temper_dict = {}
    if len(tmp) >= 9: temper_dict["리뷰"] = tmp[-3]
    temper_dict["사용자 이름"] = tmp[0]
    temper_dict["사용자 리뷰 총합"] = tmp[2]
    temper_dict["별점"] = tmp[4]
    temper_dict["리뷰 날짜"] = tmp[5]
    temper_DataFrame = pd.DataFrame(temper_dict, index=[0])
    Clean_DataFrame = Clean_DataFrame.append(temper_DataFrame, ignore_index=True)

In [56]:
Clean_DataFrame

,리뷰,사용자 이름,사용자 리뷰 총합,별점,리뷰 날짜
0,돈 개아까움 서비스 맛 뭐하나 빠지는거없이 쓰레기 홀직원 싸가지 없고,시우애비,12,3.8,2023.01.25.
1,안 좋다고 하는 것들이 진짜네요 좋은 건 주차하기 좋은 거,주하니,10,2.5,2022.12.26.
2,"알파카에 놀이방에 각종 고기들에 디저트까지... 색다르고 좋은 식당이였지만, 술먹고...",.,23,3.6,2022.11.05.
3,NaN,황신하,102,2.2,2022.11.04.
4,퍼펙트~아주좋아요~~야간 여성 직원분 친철해요^^,리~,3,4.7,2022.10.17.
5,24시라 좋구 김치가 맛있어요 동치미도,한냥충🐛,139,4.1,2022.10.04.
6,직원분들도 친절하시고 고기가 질이 좋아요 질기지 않아서 아이들도 맛있게 먹었습니다 ...,로라하,101,4.6,2022.09.22.
7,"불친절한 서비스, 순서없이 음식나오기, 시키면 함흥차사",Au revoir paris,1,1.0,2022.09.10.
8,오리고기가 좀 질기긴하네여,만물박사,32,3.5,2022.08.20.
9,NaN,ㄱㅎㄱ,"1,051",5.0,2022.08.11.


In [197]:
Clean_DataFrame.to_excel("temp.xlsx", engine="xlsxwriter", index=False)

In [ ]:
for comment in CommentURL:
    driver.get(comment)